# Python 基础

## 内建函数

在 Python3 中，`filter` 增加了延迟加载的机制，所以我们<span class="burk">必须在外面加上 `list`</span>，强制转换成 `list`，这样里面的 `lambda` 才会被执行。

In [3]:
a = [1, 2, 3, 4, 5, 6, 7, 8]
b = list(filter(lambda x: x > 3, a))
b

[4, 5, 6, 7, 8]

查看帮助。

In [6]:
#help(filter)
#help(map)

In [7]:
a = [1, 2, 3, 4]
b = list(map(lambda x: x * x, a))
b

[1, 4, 9, 16]

可以传入两个参数。

In [9]:
a = [1, 2, 3]
b = [4, 5, 6]
c = list(map(lambda x, y: x + y, a, b))
c

[5, 7, 9]

`reduce` 函数必须要使用 `functools` 函数导入。

In [12]:
from functools import reduce

a = [1, 2, 3, 4]
reduce(lambda x, y: x + y, a)

10

`zip` 函数。应用：将字段的 key 和 value 对换。

In [13]:
# 有点像矩阵转置
for i in zip((1, 2, 3), (4, 5, 6)):
    print(i)

(1, 4)
(2, 5)
(3, 6)


In [14]:
help(zip)

Help on class zip in module builtins:

class zip(object)
 |  zip(iter1 [,iter2 [...]]) --> zip object
 |  
 |  Return a zip object whose .__next__() method returns a tuple where
 |  the i-th element comes from the i-th iterable argument.  The .__next__()
 |  method continues until the shortest iterable in the argument sequence
 |  is exhausted and then it raises StopIteration.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.



有 `__iter__` 函数，表示可迭代。

应用：将 dict 的 key 和 value 对调。

In [20]:
dict1 = {'name':'liwei','city':'beijing'}
dict2 = zip(dict1.values(),dict1.keys())
dict(dict2)

{'beijing': 'city', 'liwei': 'name'}

## 闭包：内部函数使用外部变量

“内部函数使用外部变量”这种语法就叫闭包。

<span class="burk">闭包的思想来源于函数的嵌套。</span>

### 例：计算一列数的累加和。

In [21]:
def func1():
    a = 1
    b = 2
    return a + b

<span class="burk">注意</span>：
+ 下面这个函数返回了一个函数的引用；
+ <span class="mark">外部函数的变量被内部函数引用</span>。

In [22]:
def func2(a):
    def add(b):
        return a + b

    return add # 内部函数的函数名称

In [31]:
num1 = func1()
num1

3

注意：闭包要分两次执行调用，下面这个函数 10 这个变量的作用域扩大到全局了。

In [28]:
num2 = func2(10)
num2(20)

30

In [32]:
type(num1)

int

In [33]:
type(num2)

function

### 例：使用闭包实现计数器的功能。

思考：为什么 cnt 变量不能设置成 int 类型？是不是因为没有初始化

In [50]:
def counter():
    cnt = [0]

    def add_one():
        cnt[0] = cnt[0] + 1
        return cnt[0]

    return add_one

In [51]:
num1 = counter()
print(num1)

<function counter.<locals>.add_one at 0x124fd0c80>


In [52]:
print(num1())

1


In [53]:
print(num1())

2


In [54]:
print(num1())

3


下面让这个函数更加普适一些。

In [59]:
def counter(FIRST=0):
    cnt = [FIRST]

    def add_one():
        cnt[0] = cnt[0] + 1
        return cnt[0]

    return add_one

In [62]:
num2 = counter(5)
print(num2)

<function counter.<locals>.add_one at 0x124fd0400>


In [63]:
print(num2())

6


In [64]:
print(num2())

7


In [65]:
print(num2())

8


### 闭包的使用

例：输入 $a$、$b$、$x$ ，得到 $a \cdot x + b $ 的值。

In [66]:
def a_line(a, b):
    def arg_y(x):
        return a * x + b

    return arg_y

In [67]:
line1 = a_line(3, 5)
print(line1(1))

8


In [68]:
print(line1(2))

11


In [69]:
print(line1(5))

20


In [70]:
# 另起一根直线

In [73]:
line2 = a_line(10, 5)
print(line2(1))

15


In [74]:
print(line2(3))

35


## 装饰器

有一些函数我们希望它有额外的功能，可能是一些辅助功能。装饰器比较像 Java 中的过滤器，可以在一个方法的前后添加一些逻辑。

### 通过实例理解装饰器

In [78]:
import time

# 注意：这个函数是一个闭包，外部函数的参数是一个函数引用
def timer(func):
    def wrapper():
        start_time = time.time()
        # 闭包的特点 1、内部函数使用到外部变量
        func()
        end_time = time.time()
        print("函数运行了 {} 秒。".format(end_time - start_time))

    # 闭包的特点 2、外部函数返回内部函数的引用
    return wrapper

In [90]:
# 装饰器的写法其实是一个语法糖
@timer
def do_something():
    time.sleep(1)# 单位是秒
    

In [87]:
do_something()

之所以说装饰器是语法糖，是因为，我们上面写好的一个闭包 `timer` 可以通过下面的方式调用。

```python
# 注意：此时外部函数的参数是一个函数引用
func1 = timer(do_something)
func1()
```

### 让装饰器可以修饰有返回值，有参数的业务逻辑函数

首先编写业务逻辑函数。

In [91]:
def do_great_work():
    print('do_great_work begin')
    time.sleep(1)
    print('do_great_work end')

In [92]:
do_great_work()

do_great_work begin
do_great_work end


编写装饰器，其实就是一个闭包函数。

In [98]:
# 这是一个闭包函数，函数的参数是函数引用，返回内部函数引用
def display_time(func):
    def wrapper():
        t1 = time.time()
        func()  # 加上括号，表示执行这个函数
        t2 = time.time()
        print('耗时：', (t2 - t1))

    return wrapper

#### 业务函数没有返回值，没有参数

In [99]:
@display_time
def do_great_work():
    print('do_great_work begin')
    time.sleep(1)
    print('do_great_work end')
    
do_great_work()

do_great_work begin
do_great_work end
耗时： 1.003234624862671


<span class="burk">以上代码，虽然我们显示调用的是 `do_great_work()` 函数，但是实际上，执行的是闭包函数 `display_time`，解释器把 `do_great_work` 这个函数的引用传给了闭包函数。</span>

#### 业务函数有返回值，此时闭包函数怎么写



In [102]:
# 闭包函数
def display_time(func):
    def wrapper():
        t1 = time.time()
        # 1、返回值要在这里接收
        result = func()  # 加上括号，表示执行这个函数
        t2 = time.time()
        print('耗时：', (t2 - t1))
        # 2、然后再返回给上一层调用
        return result

    return wrapper

In [103]:
# 业务函数
@display_time
def do_great_work():
    print('do_great_work begin')
    time.sleep(1)
    print('do_great_work end')
    return 'well done'

In [104]:
result1 = do_great_work()
result1

do_great_work begin
do_great_work end
耗时： 1.0021789073944092


'well done'

#### 业务函数有参数，那参数要写到闭包的内部函数里面去